In [1]:
import numpy as np
import tensorflow as tf
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from CocktailEmbeddingMaker import CocktailEmbeddingMaker

class RecipeGenerationModel:
    def __init__(self, cocktail_embedding_maker, max_recipe_length=10):
        self.cocktail_embedding_maker = cocktail_embedding_maker
        self.ingredient_ids = cocktail_embedding_maker.ingredient_ids
        self.num_ingredients = cocktail_embedding_maker.num_ingredients
        self.max_recipe_length = max_recipe_length
        self.ingredient_embedding_matrix = cocktail_embedding_maker.create_ingredient_embedding_matrix()
        self.model = self.build_model()

    def build_model(self):
        model = Sequential([
            Embedding(self.num_ingredients, self.ingredient_embedding_matrix.shape[1],
                      weights=[self.ingredient_embedding_matrix], input_length=self.max_recipe_length, trainable=False),
            LSTM(128, return_sequences=True),
            LSTM(128),
            Dense(64, activation='gelu'),
            Dense(self.num_ingredients, activation='softmax')
        ])
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        return model

    def train(self, recipes, epochs=50, batch_size=32):
        ingredient_sequences = []
        next_ingredients = []

        for recipe in recipes:
            sequence = [self.ingredient_ids[self.cocktail_embedding_maker.normalize_string(ingredient)] for ingredient in recipe]
            for i in range(1, len(sequence)):
                ingredient_sequences.append(sequence[:i])
                next_ingredients.append(sequence[i])

        ingredient_sequences = tf.keras.preprocessing.sequence.pad_sequences(ingredient_sequences, maxlen=self.max_recipe_length)
        next_ingredients = tf.keras.utils.to_categorical(next_ingredients, num_classes=self.num_ingredients)

        self.model.fit(ingredient_sequences, next_ingredients, epochs=epochs, batch_size=batch_size)

    def generate_recipe(self, seed_ingredient, user_preference, max_length=10):
        generated_recipe = [seed_ingredient]
        generated_quantities = []

        for _ in range(max_length - 1):
            sequence = [self.ingredient_ids[self.cocktail_embedding_maker.normalize_string(ingredient)] for ingredient in generated_recipe]
            sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=self.max_recipe_length)

            probabilities = self.model.predict(sequence)[0]
            probabilities[sequence[0]] = 0  # 중복 재료 제거

            # 사용자 선호도를 반영하여 재료 선택 확률 조정
            for ingredient_id, prob in enumerate(probabilities):
                ingredient_name = list(self.ingredient_ids.keys())[list(self.ingredient_ids.values()).index(ingredient_id)]
                ingredient_taste_score = self.get_ingredient_taste_score(ingredient_name, user_preference)
                ingredient_abv = self.get_ingredient_abv(ingredient_name)
                abv_diff = abs(ingredient_abv - user_preference['ABV'])
                abv_score = 1 / (1 + abv_diff)  # 도수 차이가 작을수록 높은 점수
                probabilities[ingredient_id] *= ingredient_taste_score * abv_score

            next_ingredient_id = np.argmax(probabilities)
            next_ingredient = list(self.ingredient_ids.keys())[list(self.ingredient_ids.values()).index(next_ingredient_id)]
            generated_recipe.append(next_ingredient)

        # 레시피 도수 계산 및 재료 양 조정
        target_abv = user_preference['ABV']
        quantities = self.adjust_ingredient_quantities(generated_recipe, target_abv)

        return generated_recipe, quantities
    def adjust_ingredient_quantities(self, recipe, target_abv, max_iterations=100):
        quantities = [1] * len(recipe)  # 초기 재료 양 설정
        total_amount = len(recipe)

        for _ in range(max_iterations):
            recipe_abv = self.calculate_recipe_abv(recipe, quantities)

            if abs(recipe_abv - target_abv) < 0.5:  # 목표 도수와의 차이가 0.5 미만이면 종료
                break

            # 도수 차이에 따라 재료 양 조정
            if recipe_abv < target_abv:
                # 알코올 함량이 높은 재료의 양을 증가
                for i, ingredient in enumerate(recipe):
                    ingredient_info = next((item for item in self.cocktail_embedding_maker.flavor_data if item["name"] == ingredient), None)
                    if ingredient_info and ingredient_info['ABV'] > 0:
                        quantities[i] += 0.1
                        total_amount += 0.1
            else:
                # 알코올 함량이 낮은 재료의 양을 증가
                for i, ingredient in enumerate(recipe):
                    ingredient_info = next((item for item in self.cocktail_embedding_maker.flavor_data if item["name"] == ingredient), None)
                    if ingredient_info and ingredient_info['ABV'] == 0:
                        quantities[i] += 0.1
                        total_amount += 0.1

        # 총량 대비 비율로 정규화
        quantities = [q / total_amount for q in quantities]

        return quantities
    def get_ingredient_abv(self, ingredient):
        ingredient_info = next((item for item in self.cocktail_embedding_maker.flavor_data if item["name"] == ingredient), None)
        return ingredient_info['ABV'] if ingredient_info else 0

    def get_ingredient_taste_score(self, ingredient_name, user_preference):
        #ABV를 제외한 맛 특성 점수를 0~1사이로 정규화 함
        ingredient_info = next((item for item in self.cocktail_embedding_maker.flavor_data if item["name"] == ingredient_name), None)

        if ingredient_info:
            taste_scores = {taste: ingredient_info[taste] / 100 for taste in user_preference if (taste != 'ABV' and taste != 'abv_min' and taste != 'abv_max')}
            abv_score = ingredient_info['ABV'] * user_preference['ABV']
            taste_score = sum(taste_scores[taste] * user_preference[taste] for taste in taste_scores)
            return taste_score * abv_score
        else:
            return 1.0
            
    def calculate_recipe_abv(self, recipe, quantities):
        total_amount = sum(quantities)
        total_abv = 0
        for ingredient, quantity in zip(recipe, quantities):
            ingredient_info = next((item for item in self.cocktail_embedding_maker.flavor_data if item["name"] == ingredient), None)
            if ingredient_info:
                total_abv += ingredient_info['ABV'] * (quantity / total_amount)
        return total_abv


# 데이터 로드
with open('./train_data.json', 'r') as f:
    json_data = json.load(f)
with open('../flavor.json', 'r') as f:
    flavor_data = json.load(f)

# CocktailEmbeddingMaker 인스턴스 생성
cocktail_embedding_maker = CocktailEmbeddingMaker(json_data, flavor_data)

# RecipeGenerationModel 인스턴스 생성
recipe_generation_model = RecipeGenerationModel(cocktail_embedding_maker, max_recipe_length=10)

# 학습 데이터 준비
train_recipes = [recipe['recipe'].keys() for recipe in json_data['cocktail_info']]

# 모델 학습
recipe_generation_model.train(train_recipes, epochs=50, batch_size=32)


Epoch 1/50


c:\Users\aimed-lab-1\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 5.5447
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 4.9122
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.7224
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.6037
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.3576
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 4.2297
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 4.0655
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.9471
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.8007
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.6816
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.5440
Epoch 12/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.3841
Epoch 13/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.3210
Epoch 14/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.1442
Epoch 15/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.0684
Epoch 16/50
50/50 ━━━━━━━━━

In [ ]:

# 사용자 선호도 설정
user_preference = {
    'sweet': 80,
    'sour': 61,
    'bitter': 50,
    'fruity': 30,
    'ABV': 30,
    'abv_min': 0,
    'abv_max': 40
}

# 새로운 레시피 생성
seed_ingredient = 'orange juice'
generated_recipe = recipe_generation_model.generate_recipe(seed_ingredient, user_preference, max_length=5)
total_amount = 200
generated_amount = [ratio * total_amount for ratio in generated_recipe[1]]
result = []
recipe = {}
for item,quantity_ratio in zip(generated_recipe[0],generated_recipe[1]):
    recipe[item] = quantity_ratio * total_amount
    result.append(f"{item} ={quantity_ratio * total_amount}")
print(f"Generated ingredient list: {generated_recipe[0]}")
print(f"Generated Recipe : {result}")
test_abv= recipe_generation_model.calculate_recipe_abv(generated_recipe[0], generated_recipe[1])
taste_info_result = cocktail_embedding_maker.get_taste_info(recipe)
taste_info_result.pop('ID')
print(f"taste_info_result: {json.dumps(taste_info_result,indent=4)}")
print(f"user_preference: {json.dumps(user_preference,indent=4)}")
print(f"test_abv: {test_abv}")


In [22]:
# random user preference
test_user_list = []
# 'user_id', 'ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky', 'Perceived_strength'
# ABV : 0~60
# boozy : 0~100
# sweet : 0~100
# sour : 0~100
# bitter : 0~100
# umami : 0~100
attributes = ['ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky', 'Perceived_strength']
user_num = 1000
for i in range(user_num):
    user = {}
    user['user_id'] = i
    preference = {}
    user['ABV'] = np.random.randint(0,60)
    for attribute in attributes[2:]:
        user[attribute] = np.random.randint(0,100)
    test_user_list.append(user)


In [27]:
test_user_list[0]


{'user_id': 0,
 'ABV': 39,
 'sweet': 51,
 'sour': 85,
 'bitter': 11,
 'umami': 63,
 'salty': 42,
 'astringent': 51,
 'Perceived_temperature': 62,
 'spicy': 93,
 'herbal': 68,
 'floral': 33,
 'fruity': 91,
 'nutty': 47,
 'creamy': 14,
 'smoky': 67,
 'Perceived_strength': 18}

In [24]:
# evaluate
import random
total_amount = 200
seed_ingredient=random.choice(list(cocktail_embedding_maker.ingredient_ids.keys()))
print(seed_ingredient)
generated_recipe = recipe_generation_model.generate_recipe(seed_ingredient, user_preference, max_length=5)
generated_amount = [ratio * total_amount for ratio in generated_recipe[1]]
for item,quantity_ratio in zip(generated_recipe[0],generated_recipe[1]):
    recipe[item] = quantity_ratio * total_amount
test_abv= recipe_generation_model.calculate_recipe_abv(generated_recipe[0], generated_recipe[1])
taste_info_result = cocktail_embedding_maker.get_taste_info(recipe)
taste_info_result.pop('ID')
print(f"taste_info_result: {json.dumps(taste_info_result,indent=4)}")
print(f"user_preference: {json.dumps(user_preference,indent=4)}")
print(f"test_abv: {test_abv}")

carbonated water


In [ ]:

generated_recipe = recipe_generation_model.generate_recipe(seed_ingredient, user_preference, max_length=5)
total_amount = 200
generated_amount = [ratio * total_amount for ratio in generated_recipe[1]]
result = []
recipe = {}
for item,quantity_ratio in zip(generated_recipe[0],generated_recipe[1]):
    recipe[item] = quantity_ratio * total_amount
    result.append(f"{item} ={quantity_ratio * total_amount}")
print(f"Generated ingredient list: {generated_recipe[0]}")
print(f"Generated Recipe : {result}")
test_abv= recipe_generation_model.calculate_recipe_abv(generated_recipe[0], generated_recipe[1])
taste_info_result = cocktail_embedding_maker.get_taste_info(recipe)
taste_info_result.pop('ID')
print(f"taste_info_result: {json.dumps(taste_info_result,indent=4)}")
print(f"user_preference: {json.dumps(user_preference,indent=4)}")
print(f"test_abv: {test_abv}")